In [49]:
%load_ext autoreload
# %reload_ext autoreload
%autoreload 1

%aimport b6_epilepsies 
import b6_epilepsies as b6 #.py file in the same dir as this .ipynb
import seaborn as sns
import pandas as pd
from scipy.stats import mannwhitneyu

# Define the custom color palette and genotype order
new_colors_order = sns.color_palette(palette='Set2', n_colors=3)
new_colors_order[0], new_colors_order[1], new_colors_order[2] = new_colors_order[1], new_colors_order[0], new_colors_order[2]
color_map = dict(zip(['wt', 'het', 'hom'], new_colors_order))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
#Middur output NOT FOR Sleep Analysis
# Define input variables
date   = "240827"
box1   = "14"
box2   = "15"
exp    = "ALDH7A1"
export = False
    
aldh7a1_obj  = b6.MiddurData(date, box1, box2, exp, export) #.prepped_data attribute to access df
# aldh7a1_obj.quick_plot_per_fish()

Finished prepping data.


In [102]:
#Middur output NOT FOR Sleep Analysis
# Define input variables
date   = "241006"
box1   = "14"
box2   = "15"
exp    = "PNPO"
export = False
    
pnpo_obj  = b6.MiddurData(date, box1, box2, exp, export) #.prepped_data attribute to access df
# pnpo_obj.quick_plot_per_fish()

Finished prepping data.


In [101]:
middur_df = data_obj.prepped_data

# middur_df['genotype'].unique()
# # box_15 = middur_df[middur_df['box']== '15']
# # box_15
# # middur_df.to_csv('middur_df.csv')
# # middur_df




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
data = middur_df

# Function to plot data for a single box
def plot_box_data(data, box_id):
    box_data = data[data['box'] == box_id].copy()
    
    # Ensure well positions are uppercase for consistency
    box_data['well'] = box_data['well'].str.upper()

    # Count the number of unique animals for each genotype in this box
    genotype_counts = box_data.groupby('genotype')['animal'].nunique()

    # Define the 96-well plate layout
    well_order = [f"{row}{col}" for row in 'ABCDEFGH' for col in range(1, 13)]

    # Define the custom color palette and genotype order
    new_colors_order = sns.color_palette(palette='Set2', n_colors=3)
    new_colors_order[0], new_colors_order[1], new_colors_order[2] = (
        new_colors_order[1],
        new_colors_order[0],
        new_colors_order[2],
    )
    color_map = dict(zip(['wt', 'het', 'hom'], new_colors_order))

    # Initialize the figure and axes for the grid
    sns.set_theme(style="whitegrid")
    fig, axes = plt.subplots(8, 12, figsize=(20, 15), sharex=True, sharey=True)
    fig.suptitle(
        f"Activity Traces for Zebrafish in Box {box_id} (96-Well Plate Layout)",
        fontsize=16,
    )

    # Iterate through each subplot (well position) and plot data if available
    for i, well in enumerate(well_order):
        row, col = divmod(i, 12)
        ax = axes[row, col]

        well_data = box_data[box_data['well'] == well]
        if not well_data.empty:
            genotype = well_data['genotype'].iloc[0] if 'genotype' in well_data else 'wt'
            color = color_map.get(genotype, 'gray')  # Default to gray if genotype is missing
            ax.plot(well_data['start'], well_data['middur'], color=color, label=f"{genotype.upper()}")
            ax.set_title(well, fontsize=8)
        
        # Turn off axis for cleaner presentation
        # ax.axis('off')

    # Add faint lines to outline the plate boundary
    for ax in axes[:, 0]:
        ax.spines['left'].set_color('lightgrey')
        ax.spines['left'].set_linewidth(0.5)
    for ax in axes[-1, :]:
        ax.spines['bottom'].set_color('lightgrey')
        ax.spines['bottom'].set_linewidth(0.5)

    # Add a legend for genotype colors with counts
    handles = [
        plt.Line2D(
            [0], [0], color=color, lw=2,
            label=f"{genotype.upper()} ({genotype_counts.get(genotype, 0)})"
        )
        for genotype, color in color_map.items()
    ]
    fig.legend(handles=handles, loc='upper right', title="Genotype (Count)")

    # Adjust layout for better visualization
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Get unique boxes from the dataset
unique_boxes = data['box'].unique()

# Plot data for each box
for box_id in unique_boxes:
    plot_box_data(data, box_id)

In [18]:
# import pandas as pd

start = box_15['stdate_sttime'].min()
end   = box_15['stdate_sttime'].max()

# Calculate the starting timestamp for the last 9/24 hours
last_9_hours_start = end - pd.Timedelta(hours=9)
# last_24_hours_start = end - pd.Timedelta(hours=24)

# Filter data for the last 9 hours
filtered_last9h_df_15 = box_15[box_15['stdate_sttime'] > last_9_hours_start]
last9h_df_groupedby_geno_time_15 = filtered_last9h_df_15.groupby(['genotype','stdate_sttime'])['middur'].agg(['mean', 'median', 'std', 'min', 'max', 'count','sum'])

# Filter data for the last 24 hours
# filtered_last24h_df_15 = box_15[box_15['stdate_sttime'] > last_24_hours_start]
# last24h_df_groupedby_geno_time_15 = filtered_last24h_df_15.groupby(['genotype','stdate_sttime'])['middur'].agg(['mean', 'median', 'std', 'min', 'max', 'count','sum'])

data = filtered_last9h_df_15

genotype_stats_data = data.groupby(['animal','genotype'])['middur'].agg(['mean', 'median', 'std', 'min', 'max', 'count', 'sum'])


print('Last 9 hours, Box 15')
print(' ')
# Print the recalculated stats
print(genotype_stats_data)
print(' ')

# Step 4 (Optional): Perform ANOVA and t-tests again
from scipy.stats import f_oneway, ttest_ind

# ANOVA for the last day
groups = [group['middur'].values for _, group in data.groupby(['animal','genotype'])]

anova_result = f_oneway(*groups)
print('ANOVA F-statistic:', anova_result.statistic)
print('ANOVA p-value:', anova_result.pvalue)
print(' ')

# Pairwise t-tests for the last day
# genotypes = data['genotype'].unique()
# for g1, g2 in combinations(genotypes, 2):
#     group1 = data[data['genotype'] == g1]['middur']
#     group2 = data[data['genotype'] == g2]['middur']
#     ttest_result_last_day = ttest_ind(group1, group2)
#     print(f'T-test between {g1} and {g2}: p-value={ttest_result_last_day.pvalue}')

# NON PARAMETRIC
# Perform the Mann-Whitney U test (median)

# for g1, g2 in combinations(genotypes, 2):
#     group1 = data[data['genotype'] == g1]['middur']
#     group2 = data[data['genotype'] == g2]['middur']
#     stat, p = mannwhitneyu(group1, group2, alternative='two-sided')

#     print(' ')
#     print(f"U statistic: {stat}")
#     print(f"P-value: {p}")
    
#     # Interpretation
#     alpha = 0.05
#     if p < alpha:
#         print("Reject the null hypothesis: The distributions are different.")
#     else:
#         print("Fail to reject the null hypothesis: No significant difference in distributions.")
        

Last 9 hours, Box 15
 
                     mean  median       std  min    max  count       sum
animal genotype                                                         
2-A01  het       0.641593  0.0490  1.102592  0.0  5.811    540   346.460
2-A02  het       0.646546  0.2215  0.866246  0.0  4.481    540   349.135
2-A04  het       0.040661  0.0000  0.222169  0.0  2.397    540    21.957
2-A05  wt        0.654926  0.0410  1.033926  0.0  5.432    540   353.660
2-A06  het       1.708837  1.6685  1.103698  0.0  6.372    540   922.772
2-A07  wt        1.679861  1.7290  1.034825  0.0  5.734    540   907.125
2-A09  het       0.512761  0.1230  0.798860  0.0  4.475    540   276.891
2-A10  het       0.039320  0.0000  0.264646  0.0  3.600    540    21.233
2-A11  het       1.207411  1.1465  1.037504  0.0  5.873    540   652.002
2-A12  hom       1.701896  1.2230  1.564538  0.0  8.765    540   919.024
2-B01  wt        0.180454  0.0000  0.494947  0.0  4.921    540    97.445
2-B02  het       2.339870  1

In [15]:
genotype_count

start                                                    \
                   count      mean            std  min       25%       50%   
genotype animal                                                              
het      2-A01   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A02   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A04   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A06   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A09   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A10   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A11   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B02   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C05   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C08   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C12   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D08   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D09   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D10   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D11   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D12   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E01   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E03   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E06   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
hom      2-A12   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B11   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C02   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C03   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C09   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C10   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E08   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E09   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
wt       2-A05   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-A07   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B01   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B03   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B05   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B08   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-B12   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-C11   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D02   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-D04   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   
         2-E02   14704.0  441090.0  254689.410852  0.0  220545.0  441090.0   

                                         end                 ...   middur  \
                      75%       max    count           mean  ...      75%   
genotype animal                                              ...            
het      2-A01   661635.0  882180.0  14704.0  441149.997544  ...  2.40600   
         2-A02   661635.0  882180.0  14704.0  441149.997544  ...  1.28125   
         2-A04   661635.0  882180.0  14704.0  441149.997544  ...  2.04325   
         2-A06   661635.0  882180.0  14704.0  441149.997544  ...  2.66700   
         2-A09   661635.0  882180.0  14704.0  441149.997544  ...  3.03050   
         2-A10   661635.0  882180.0  14704.0  441149.997544  ...  1.79700   
         2-A11   661635.0  882180.0  14704.0  441149.997544  ...  1.82025   
         2-B02   661635.0  882180.0  14704.0  4411

The dataset includes the relevant columns for genotype, animal, and activity (middur). Below is a Python script to analyze whether there are significant differences in middur activity between genotypes using the following statistical tests:
	1.	Shapiro-Wilk test for normality.
	2.	Levene’s test for homogeneity of variances.
	3.	One-way ANOVA if assumptions are met.
	4.	Kruskal-Wallis test if assumptions are not met.
	5.	Post-hoc tests for pairwise comparisons.

In [11]:
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Load the dataset
# file_path = 'filtered_last9h_df_15.csv'  # Update the path if needed
data = filtered_last9h_df_15

# Select relevant columns
data = data[['animal','genotype', 'middur']]

# Group data by genotype
grouped = data.groupby(['animal','genotype'])['middur']

# Perform Shapiro-Wilk test for normality
print("Shapiro-Wilk Test for Normality:")
for genotype, values in grouped:
    stat, p = stats.shapiro(values)
    print(f"{genotype}: W={stat:.4f}, p={p:.4f}")

# Perform Levene's test for homogeneity of variances
stat, p = stats.levene(*[values for _, values in grouped])
print(f"\nLevene's Test for Homogeneity of Variances: W={stat:.4f}, p={p:.4f}")

# Determine the appropriate test based on assumptions
if all(stats.shapiro(values)[1] > 0.05 for _, values in grouped) and p > 0.05:
    # Perform one-way ANOVA
    stat, p = stats.f_oneway(*[values for _, values in grouped])
    print(f"\nOne-Way ANOVA: F={stat:.4f}, p={p:.4f}")

    # Post-hoc test (Tukey's HSD)
    if p < 0.05:
        tukey = pairwise_tukeyhsd(data['middur'], data['genotype'])
        print("\nPost-hoc Tukey HSD Test:")
        print(tukey)
else:
    # Perform Kruskal-Wallis test
    stat, p = stats.kruskal(*[values for _, values in grouped])
    print(f"\nKruskal-Wallis Test: H={stat:.4f}, p={p:.5f}")
    print('The Kruskal-Wallis test, a non parametric')

    # Post-hoc test (Dunn's test with Bonferroni correction)
    if p < 0.05:
        from scikit_posthocs import posthoc_dunn
        posthoc = posthoc_dunn(data, val_col='middur', group_col='genotype', p_adjust='bonferroni')
        print("\nPost-hoc Dunn Test with Bonferroni Correction:")
        print(posthoc)

# Summary: Save results to a CSV file if needed
data.describe()
# data.describe().to_csv('middur_activity_summary.csv')

Shapiro-Wilk Test for Normality:
('2-A01', 'het'): W=0.6486, p=0.0000
('2-A02', 'het'): W=0.7713, p=0.0000
('2-A04', 'het'): W=0.1826, p=0.0000
('2-A05', 'wt'): W=0.6957, p=0.0000
('2-A06', 'het'): W=0.9712, p=0.0000
('2-A07', 'wt'): W=0.9714, p=0.0000
('2-A09', 'het'): W=0.6926, p=0.0000
('2-A10', 'het'): W=0.1361, p=0.0000
('2-A11', 'het'): W=0.9189, p=0.0000
('2-A12', 'hom'): W=0.8865, p=0.0000
('2-B01', 'wt'): W=0.4183, p=0.0000
('2-B02', 'het'): W=0.8686, p=0.0000
('2-B03', 'wt'): W=0.9660, p=0.0000
('2-B05', 'wt'): W=0.5657, p=0.0000
('2-B08', 'wt'): W=0.5417, p=0.0000
('2-B11', 'hom'): W=0.7146, p=0.0000
('2-B12', 'wt'): W=0.8009, p=0.0000
('2-C02', 'hom'): W=0.9290, p=0.0000
('2-C03', 'hom'): W=0.9964, p=0.2652
('2-C05', 'het'): W=0.2132, p=0.0000
('2-C08', 'het'): W=0.9136, p=0.0000
('2-C09', 'hom'): W=0.6410, p=0.0000
('2-C10', 'hom'): W=0.8636, p=0.0000
('2-C11', 'wt'): W=0.4320, p=0.0000
('2-C12', 'het'): W=0.9128, p=0.0000
('2-D02', 'wt'): W=0.9366, p=0.0000
('2-D04', 'wt'

,middur
count,20520.000000
mean,1.077807
std,1.426513
min,0.000000
25%,0.000000
50%,0.398000
75%,1.806250
max,9.700000


In [19]:
import pandas as pd
import scipy.stats as stats
import scikit_posthocs as sp



# Select relevant columns
filtered_data = data[['stdate_sttime','genotype', 'middur']]

# Group data by genotype
grouped = filtered_data.groupby(['genotype','stdate_sttime'])['middur']

# Perform Kruskal-Wallis test
kruskal_stat, kruskal_p = stats.kruskal(*[values for _, values in grouped])
print("Kruskal-Wallis Test Results:")
print(f"H-statistic: {kruskal_stat:.4f}")
print(f"p-value: {kruskal_p:.4e}")

# Perform pairwise post-hoc Dunn test with Bonferroni correction
if kruskal_p < 0.05:
    posthoc_results = sp.posthoc_dunn(filtered_data, val_col='middur', group_col='genotype', p_adjust='bonferroni')
    print("\nPairwise Post-hoc Dunn Test Results (Bonferroni Adjusted):")
    print(posthoc_results)
else:
    print("\nNo significant differences detected; post-hoc analysis not performed.")

Kruskal-Wallis Test Results:
H-statistic: 2585.0067
p-value: 1.2745e-47

Pairwise Post-hoc Dunn Test Results (Bonferroni Adjusted):
               het            hom             wt
het   1.000000e+00  5.159043e-111   1.142961e-09
hom  5.159043e-111   1.000000e+00  1.792027e-142
wt    1.142961e-09  1.792027e-142   1.000000e+00
